# Figures

In [0]:
cd ~/data/landsat

In [0]:
import warnings
warnings.filterwarnings('ignore')

In [0]:
plt = plotter(backend='agg')
%matplotlib inline

In [0]:
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000
from pyproj import Proj
from matplotlib import cm

In [0]:
import location_tools

In [0]:
save = True

In [0]:
pixel = 15
cmap = cm.jet
extent = lambda s: [-pixel*s/2, pixel*s/2, -pixel*s/2, pixel*s/2]
decloud = lambda x, cut: np.choose(x<cut, [np.zeros_like(x), x])
amp = lambda x, alpha: 255*((x-np.min(x))/(np.max(x)-np.min(x)))**alpha

## CGS-02 and BD-09 Correction

In [0]:
index = pd.read_csv('targets/google_scenes_2002_mincloud.csv')

In [0]:
x_lon, x_lat = 116.390797, 39.916406 # beijing (forbidden city)

# scale params
vmin, vmax = 32, 100
rad, size = 256, 512
ext = extent(size)

# extract tiles
tile0 = location_tools.extract_satelite_tile(x_lon, x_lat, rad, index=index, size=size, proj='bd-09')
tile1 = location_tools.extract_satelite_tile(x_lon, x_lat, rad, index=index, size=size, proj='wgs-84')
arr0, arr1 = np.asarray(tile0), np.asarray(tile1)

# make plots
fig, ax = plt.subplots(ncols=2, figsize=(12, 6))
ax[0].imshow(arr0, cmap=cm.jet, interpolation='lanczos', vmin=vmin, vmax=vmax, extent=ext)
ax[1].imshow(arr1, cmap=cm.jet, interpolation='lanczos', vmin=vmin, vmax=vmax, extent=ext)
ax[0].scatter([0], [0], c='r');
ax[1].scatter([0], [0], c='r');
ax[0].set_title('Uncorrected BG-09 Coordinates')
ax[1].set_title('Corrected WGS-84 Coordinates')
if save: fig.savefig('docs/images/bd_wgs_compare.png', dpi=300);

## Landsat Imagery Demo

In [0]:
coldefs = ['id', 'lon_bd09', 'lat_bd09']
firms = pd.read_csv('firms/census_2004_geocode.csv', usecols=coldefs)

In [0]:
fid = 1082781 # Shanghai Happy Vacuum Cleaner Factory
info = firms[firms['id']==fid].iloc[0]
x_lon, x_lat = info[['lon_bd09', 'lat_bd09']]

# scale params
rad0, rad1 = 256, 1024
vmin, vmax = 32, 130
ext0, ext1 = extent(rad0), extent(rad1)
size = 512

# extract tiles
tile0 = location_tools.extract_satelite_tile(x_lon, x_lat, rad=rad0, index=index, size=size, proj='bd-09')
tile1 = location_tools.extract_satelite_tile(x_lon, x_lat, rad=rad1, index=index, size=size, proj='bd-09')
arr0, arr1 = np.asarray(tile0), np.asarray(tile1)
arr0, arr1 = decloud(arr0, 100), decloud(arr1, 100)

# make plots
fig, ax = plt.subplots(ncols=2, figsize=(12, 6))
ax[0].imshow(arr0, cmap=cmap, interpolation='lanczos', vmin=vmin, vmax=vmax, extent=ext0)
ax[1].imshow(arr1, cmap=cmap, interpolation='lanczos', vmin=vmin, vmax=vmax, extent=ext1)
ax[0].scatter([0], [0], c='r');
ax[1].scatter([0], [0], c='r');
ax[0].set_title('Firm Location, 2km Radius')
ax[1].set_title('Firm Location, 7km Radius')
if save: fig.savefig('docs/images/landsat_demo.png', dpi=300);

## Firm Density Demo

In [0]:
# lon1, lat1 = 116.390797, 39.916406 # beijing
# lon1, lat1 = 114.066500, 22.654437 # huawei
# lon1, lat1 = 104.066465, 30.658427 # chengdu
lon0, lat0 = 121.439610, 31.265273 # shanghai
lon1, lat1 = 108.944372, 34.265481 # xi'an

rad0, rad1 = 1800, 512
ext0, ext1 = extent(rad0), extent(rad1)
vmin, vmax = 32, 130
size = 512

sat = location_tools.extract_satelite_tile(lon0, lat0, rad=rad0, index=index, size=size, proj='bd-09')
den = location_tools.extract_density_tile(lon0, lat0, rad=rad0, size=size, proj='bd-09')
arr_sat, arr_den = np.asarray(sat), np.asarray(den)
arr_sat, arr_den = 1.0*amp(decloud(arr_sat, 100), 2.5), amp(arr_den, 0.7)
arr_mix0 = np.stack([arr_sat/255+2*arr_den/255, arr_sat/255, arr_sat/255], axis=2)

sat = location_tools.extract_satelite_tile(lon1, lat1, rad=rad1, index=index, size=size, proj='bd-09')
den = location_tools.extract_density_tile(lon1, lat1, rad=rad1, size=size, proj='bd-09')
arr_sat, arr_den = np.asarray(sat), np.asarray(den)
arr_sat, arr_den = 1.4*amp(decloud(arr_sat, 100), 3.5), amp(arr_den, 0.7)
arr_mix1 = np.stack([arr_sat/255+2*arr_den/255, arr_sat/255, arr_sat/255], axis=2)

fig, ax = plt.subplots(ncols=2, figsize=(12, 6))
ax[0].imshow(arr_mix0, interpolation='lanczos', extent=ext0)
ax[1].imshow(arr_mix1, interpolation='lanczos', extent=ext1)
ax[0].set_title('Shanghai, 13k Radius')
ax[1].set_title('Xi\'an, 4km Radius')
if save: fig.savefig('docs/images/density_overlay.png', dpi=300);

## Density Maps

In [0]:
def chunk_path(tag, loc='tiles/density/1024px', ext='jpg'):
    tag = f'{tag:07d}'
    sub = tag[:4]
    psub = f'{loc}/{sub}'
    ptag = f'{psub}/{tag}.{ext}'
    return ptag

In [0]:
def add_border(x):
    x = x.copy()
    x[0,:] = 255
    x[-1,:] = 255
    x[:,0] = 255
    x[:,-1] = 255
    return x

In [0]:
# ids = [425286, 425777, 425920, 425951]
ids = [1082781, 425777, 39676, 244861]

In [0]:
paths_den = [chunk_path(i, loc='tiles/density/1024px') for i in ids]
images_den = [np.asarray(Image.open(p)) for p in paths_den]
images_den = [255*(x>40) for x in images_den]
# images_den = [amp(x, 0.7) for x in images_den]
images_den = [x.astype(np.uint8) for x in images_den]
images_den = [add_border(x) for x in images_den]

In [0]:
fig, axs = plt.subplots(ncols=len(ids), figsize=(12, 3))
for i, _ in enumerate(ids):
    ax = axs[i]
    ax.imshow(images_den[i], origin='upper', cmap='Greys', interpolation='lanczos')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.axis('off')
if save: fig.savefig('docs/images/density_examples.png', dpi=300, bbox_inches='tight')

In [0]:
paths_sat = [chunk_path(i, loc='tiles/landsat/1024px') for i in ids]
images_sat = [np.asarray(Image.open(p)) for p in paths_sat]
images_sat = [amp(x, 0.2) for x in images_sat]
images_sat = [x.astype(np.uint8) for x in images_sat]
images_sat = [add_border(x) for x in images_sat]

In [0]:
fig, axs = plt.subplots(ncols=len(ids), figsize=(12, 3))
for i, _ in enumerate(ids):
    ax = axs[i]
    ax.imshow(images_sat[i], origin='lower', cmap='Greys', interpolation='lanczos')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.axis('off')
if save: fig.savefig('docs/images/landsat_examples.png', dpi=300, bbox_inches='tight')

In [0]:
id1 = 1082781

path_den = chunk_path(id1, loc='tiles/density/1024px')
image_den0 = np.asarray(Image.open(path_den))
image_den = 3*image_den0
image_den = image_den.astype(np.uint8)
image_den = add_border(image_den)

path_sat = chunk_path(id1, loc='tiles/landsat/1024px')
image_sat0 = np.asarray(Image.open(path_sat))
image_sat = amp(image_sat0, 0.2)
image_sat = image_sat.astype(np.uint8)
image_sat = add_border(image_sat)

image_sim = 255*((image_sat>150)&(image_sat<155))
image_sim = image_sim.astype(np.uint8)
image_sim = add_border(image_sim)

fig, ax = plt.subplots(figsize=(3, 3))
ax.imshow(image_den, origin='upper', cmap='Greys')
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
ax.axis('off')
if save: fig.savefig('docs/images/density_single.png', dpi=300, bbox_inches='tight')

fig, ax = plt.subplots(figsize=(3, 3))
ax.imshow(image_sat, origin='upper', cmap='Greys')
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
ax.axis('off')
if save: fig.savefig('docs/images/landsat_single.png', dpi=300, bbox_inches='tight')

fig, ax = plt.subplots(figsize=(3, 3))
ax.imshow(image_sim, origin='upper', cmap='Greys')
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
ax.axis('off')
if save: fig.savefig('docs/images/simdens_single.png', dpi=300, bbox_inches='tight')